In [1]:
import polars as pl
import datetime as dt

df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

df

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


## JSON

In [2]:
from io import StringIO

json_str = '[{"foo":1,"bar":6,"e":{"b":1,"f":{"c":2}}},{"foo":2,"bar":7,"e":{"b":2}},{"foo":3,"bar":8,"e":{"b":3}}]'
df = pl.read_json(StringIO(json_str))
df

foo,bar,e
i64,i64,struct[2]
1,6,"{1,{2}}"
2,7,"{2,null}"
3,8,"{3,null}"


In [3]:
df.select(pl.col('e').struct.field('b')).sort('b')

b
i64
1
2
3


In [4]:
df.unnest('e')

foo,bar,b,f
i64,i64,i64,struct[1]
1,6,1,{2}
2,7,2,null
3,8,3,null


In [5]:
df.unnest('e').unnest('f')

foo,bar,b,c
i64,i64,i64,i64
1,6,1,2
2,7,2,null
3,8,3,null
